# <a id='toc1_'></a>[Description](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Description](#toc1_)    
- [Imports](#toc2_)    
- [Examples](#toc3_)    
  - [Simple](#toc3_1_)    
  - [With conversation history](#toc3_2_)    
  - [With direct agent executor for more control](#toc3_3_)    
  - [With consistency search](#toc3_4_)    
  - [With custom callback](#toc3_5_)    
  - [With LangSmith tracing](#toc3_6_)    
  - [Simple check of list of vendors wether they are medical equipment vendors](#toc3_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Agentic approach to information checking

# <a id='toc2_'></a>[Imports](#toc0_)

In [1]:
import os
import sys

# Change working directory to parent (project root)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# Ensure parent directory is in sys.path so that pyagent_smith is importable
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [ ]:
import warnings
from urllib3.exceptions import InsecureRequestWarning

warnings.filterwarnings("ignore", category=InsecureRequestWarning)

import pandas as pd
from IPython.display import display


from pyagent_smith.agent import build_chat_agent

import json
import re
from typing import List
import json5

In [3]:
# Set display options to show all rows
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
# pd.set_option("display.width", None)

# Optionally, reset display options after printing
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')

# <a id='toc3_'></a>[Examples](#toc0_)

## <a id='toc3_1_'></a>[Simple search](#toc0_)

In [4]:
def check_if_there_is_rock_climbing_around_cellecs():
    agent = build_chat_agent(
        system_prompt=(
            "You are a helpful assistant that researches locations and determines "
            "if there is any rock climbing available around them or not. "
            "Use web search to find accurate information about rock climbing locations."
        ),
        verbose=True,
    )

    if agent is None:
        return "Error: Ollama not configured"

    query = (
        "Is there any rock climbing around Cellecs, Catalunya?"
        "Search for information about rock climbing locations around Cellecs."
    )

    result = agent.invoke(query)

    print(f"Query: {query}")
    print(f"Result: {result}")

In [5]:
check_if_there_is_rock_climbing_around_cellecs()

[Tool] web_search started
[Tool] Input: {'max_results': '5', 'query': 'rock climbing locations near Cellecs, Catalunya'}
[Tool] Output: content="1. Catalunya – Rockfax\n   URL: https://rockfax.com/climbing-guides/rockfax-digital/catalunya/\n   In addition, are seven smaller crags situated near Barcelona and a couple more near Lleida.Barcelona Area Cellecs , Gelida, La Facu, La Font de l'Ametlló, Pas de la Mala Dona, Penya Ginesta, Subirats, Siurana. Lleida Area Ager, Cubells (free sample), Terradets, Vilanova de Meià.\n\n2. Cellecs | RockAroundTheWorld\n   URL: https://rockaroundtheworld.co.uk/2018/03/12/cellecs/\n   Plaça Cellecs is the best bit of rock ; a 20m high, easy angled, slab of crystalline granite with some knobbly holds and generally good friction.Tags: Barcelona, Catalunya , Cellecs , escalada., rock climbing . Post navigation.\n\n3. How to Do Rock Climbing ( Locations ) - Sims 4 - YouTube\n   URL: https://www.youtube.com/watch?v=cX2wysfh1BI\n   In this video, we will be s

## <a id='toc3_2_'></a>[With conversation history](#toc0_)

In [6]:
def check_with_conversation_history():
    """Check with automatic conversation history for follow-up questions."""
    agent = build_chat_agent(
        system_prompt=(
            "You are a helpful assistant that researches locations and determines "
            "if there is any rock climbing available around them or not. "
            "Use web search to find accurate information about rock climbing locations."
        ),
        retain_chat_history=True,  # Enable automatic history retention
        verbose=True
    )

    if agent is None:
        return "Error: Ollama not configured"

    # First query - history is automatically maintained
    query1 = "Is there any rock climbing around Cellecs, Catalunya?"
    result1 = agent.invoke(query1)
    print(f"First query: {query1}")
    print(f"First response: {result1}")

    # Follow-up query - agent automatically uses previous conversation context
    query2 = "List all locations around."
    result2 = agent.invoke(query2)
    print(f"Follow-up query: {query2}")
    print(f"Follow-up response: {result2}")

    # print(f"agent.history: {agent.history}")
    return agent


In [7]:
agent = check_with_conversation_history()

[Tool] web_search started
[Tool] Input: {'max_results': '10', 'query': 'rock climbing near Cellecs, Catalonia'}
[Tool] Output: content="1. Rock Climbing near Barcelona - Laid-back places\n   URL: https://laidbackplaces.com/rock-climbing-near-barcelona/\n   September 20, 2022 - We climbed in Sector Placa (mostly slab, grades from 4 – 6b) and Sector Agulla de Cellecs (slab, a few short routes with roofs to practice aid climbing). Sant Llorenç de Munt is a popular hiking and climbing area and only a 30-40 minutes’ ...\n\n2. Laid-back places - Best rock climbing in Catalunya and other places\n   URL: https://laidbackplaces.com/\n   It is a very popular sport climbing destination. It is especially attractive during the hotter months thanks to its canyons with crystal clear water where you can enjoy a […] ... abandoned buildings adventure agriculture austria barcelona beach beaches boulder castle catalonia caves city cliff jumping cliffs climbing climbing wall costa brava costa dorada coves 

In [9]:
agent.chat_history

[HumanMessage(content='Is there any rock climbing around Cellecs, Catalunya?', additional_kwargs={}, response_metadata={}, id='2b5d7e3a-cfdc-498d-a63c-2768d6aff476'),
 AIMessage(content="Yes, there are many rock climbing locations around Cellecs, Catalunya. Some popular options include:\n\n* Sector Placa and Sector Agulla de Cellecs in the Sant Llorenç de Munt area\n* The Catalan coastal ranges north, which offer a variety of sport climbing routes\n* Montserrat, a popular hiking and climbing destination with stunning views\n* Garraf and Penedes, both of which have a range of rock climbing routes\n\nThese locations offer a range of climbing styles and difficulties, from beginner-friendly slab routes to more challenging sport climbs. It's worth noting that some of these areas may require permission or guides to access.\n\nIt's also worth mentioning that Catalonia is known for its excellent rock climbing conditions, with mild winters and warm summers making it an ideal destination for cli

## <a id='toc3_4_'></a>[With consistency search and custom callback](#toc0_)

* i.e. multiple web search engines are used to find the most consistent answer https://www.promptingguide.ai/techniques/consistency

In [10]:
def check_with_self_consistency_search():
    """Use self-consistency search with majority voting for more reliable results."""
    agent = build_chat_agent(
        system_prompt=(
            "You are a helpful assistant that researches locations and determines "
            "if there is any rock climbing available around them or not. "
            "Use web search to find accurate information about rock climbing locations."
        ),
        use_self_consistency_search=True,  # Enable self-consistency with majority voting
        verbose=True,
    )

    if agent is None:
        return "Error: Ollama not configured"

    query = "Is Cellecs in Catalunya a climbing location?"
    result = agent.invoke(query)

    print(f"Query: {query}")
    print(f"Result: {result}")
    return result

In [11]:
result = check_with_self_consistency_search()

[Tool] web_search_with_self_consistency started
[Tool] Input: {'max_results': '5', 'query': 'Callecs in Catalunya rock climbing'}
[Tool] Output: content="1. Home - Climb Catalunya\n   URL: https://climbcatalunya.com/\n   Climb Catalunya organizes dedicated climbing trips to some of Europe's most dramatic rock climbing destinations. Relative novices and hardened enthusiasts alike can enjoy our hassle free holidays and long weekend climbing breaks. For those looking to improve their skills or climb their dream ...\n   (Consensus: 3/3 engines)\n\n2. Rock Climbing in Catalonia, Catalonia\n   URL: https://www.mountainproject.com/area/106002779/catalonia\n   Catalonia is one of the best places to climb in Spain. Plenty of sport crags known worldwide and lots of trad/adventure climbing areas. You can find different types of rock, but the most usual is limestone, with some really particular and nice conglomerate areas like the mystical Montserrat.\n   (Consensus: 3/3 engines)\n\n3. THE 15 BEST

## <a id='toc3_5_'></a>[Calculator tool with custom callback wo verbosity](#toc0_)

In [12]:
def check_with_custom_callbacks():
    """Example using custom callbacks for monitoring agent execution."""
    from langchain_core.callbacks import BaseCallbackHandler

    class SimpleCallbackHandler(BaseCallbackHandler):
        def on_tool_start(self, serialized, input_str, **kwargs):
            print(f"Tool started: {serialized.get('name', 'unknown')}")

        def on_tool_end(self, output, **kwargs):
            print(f"Tool completed. Output length: {len(str(output))}")

    agent = build_chat_agent(
        system_prompt="You are a helpful assistant.",
        callbacks=[SimpleCallbackHandler()],
    )

    if agent is None:
        return "Error: Ollama not configured"

    result = agent.invoke("What is 2+2? Use the calculator tool.")
    print(f"Query: What is 2+2? Use the calculator tool.")
    print(f"Result: {result}")

In [13]:
check_with_custom_callbacks()

Tool started: calculator
Tool completed. Output length: 81
Query: What is 2+2? Use the calculator tool.
Result: The answer to your question is 4.


## <a id='toc3_6_'></a>[Simple check of list of location wether there is any climbing around](#toc0_)

In [11]:
def parse_json_response(response_text: str) -> List[dict]:
    """Parse JSON response from agent."""
    json_match = re.search(r"```json\s*(.*?)\s*```", response_text, re.DOTALL)
    if json_match:
        response_text = json_match.group(1).strip()
    else:
        response_text = response_text.replace("```json", "").replace("```", "").strip()

    if response_text.strip().startswith("["):
        return json5.loads(response_text)
    elif response_text.strip().startswith("{"):
        return [json5.loads(response_text)]
    else:
        json_match = re.search(r"\[.*\]", response_text, re.DOTALL)
        if json_match:
            return json5.loads(json_match.group(0))
        raise ValueError("No valid JSON found")


def simple_climbing_locations_check(locations: List[str]) -> dict:
    """
    Simple example: Check if location is a climbing location using web search.

    Args:
        locations: List of location names

    Returns:
        dict with 'results' (list of responses) and 'all_received' (bool)
    """
    # Create agent with web search capability
    agent = build_chat_agent(
        system_prompt=(
            "You are a helpful assistant that researches locations and determines "
            "if there is any rock climbing available around them or not."
            "Use web search to find accurate information about rock climbing locations."
        ),
        retain_chat_history=True,
        verbose=True,
    )

    if agent is None:
        return {"error": "Agent not available", "results": [], "all_received": False}

    # Build prompt with explicit instruction to use web search
    schema = {
        "type": "object",
        "properties": {
            "location": {"type": "string"},
            "is_rock_climbing_location": {"type": "boolean"},
            "explanation": {"type": "string"},
            "web_search_results": {"type": "string"},
        },
        "required": ["location", "is_rock_climbing_location", "explanation", "web_search_results"],
    }

    location_list = "\n".join([f"  - {v}" for v in locations])
    prompt = (
        f"For each location in the list below, use the web_search tool to research whether "
        f"they are a rock climbing location or not.\n\n"
        f"CRITICAL: When using the web_search tool, you MUST pass a search query STRING as the 'query' parameter. "
        f"For example, for location 'Montserrat', call: web_search(query='rock climbing in Montserrat', max_results=5). "
        f"DO NOT pass JSON objects, schemas, or result fields to web_search - only pass search query strings like 'rock climbing in [location name]'.\n\n"
        f"Location list:\n{location_list}\n\n"
        f"Workflow:\n"
        f"1. For each location, call web_search with a query string like 'rock climbing in [location name]'\n"
        f"2. After getting search results for all locations, format your response as a JSON array\n"
        f"3. Each entry in the array should follow this format:\n"
        f"{json.dumps(schema, indent=2)}\n\n"
        f"Return only the JSON array, no other text."
    )

    # Call agent
    print(f"Prompt:\n {prompt}\n")
    response = agent.invoke(prompt)

    # Parse response
    try:
        results = parse_json_response(response)
        print(f"Results:\n {json.dumps(results, indent=2)}")
    except Exception as e:
        return {"error": str(e), "results": [], "all_received": False}

In [ ]:
locations = ["Montserrat", "Siurana", "Margalef", "Garraf"]
simple_climbing_locations_check(locations)

Prompt:
 For each location in the list below, use the web_search tool to research whether they are a rock climbing location or not.

CRITICAL: When using the web_search tool, you MUST pass a search query STRING as the 'query' parameter. For example, for location 'Montserrat', call: web_search(query='rock climbing in Montserrat', max_results=5). DO NOT pass JSON objects, schemas, or result fields to web_search - only pass search query strings like 'rock climbing in [location name]'.

Location list:
  - Montserrat
  - Siurana
  - Margalef
  - Garraf

Workflow:
1. For each location, call web_search with a query string like 'rock climbing in [location name]'
2. After getting search results for all locations, format your response as a JSON array
3. Each entry in the array should follow this format:
{
  "type": "object",
  "properties": {
    "location": {
      "type": "string"
    },
    "is_rock_climbing_location": {
      "type": "boolean"
    },
    "explanation": {
      "type": "strin